In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


In [3]:
#importing packages

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import image_dataset_from_directory
import warnings

warnings.filterwarnings('ignore')

In [30]:
#checking for duplications

import os
def get_all_image_paths(folder_path):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.jpg')):
                image_paths.append(os.path.join(root, file))
    return set(image_paths)

train_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Train')
val_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Validation')
test_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Test')

# Intersections
print("Train ∩ Val:", len(train_paths & val_paths))
print("Train ∩ Test:", len(train_paths & test_paths))
print("Val ∩ Test:", len(val_paths & test_paths))

Train ∩ Val: 0
Train ∩ Test: 0
Val ∩ Test: 0


In [4]:
#loading image data
def load_1stimage_data(path):
    img_size = (256, 256)
    batch_size = 64

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Train",
        label_mode="binary",  
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Validation",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True  
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Test",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False  
    )


    return train_ds,val_ds ,test_ds

train,val,test=load_1stimage_data('/kaggle/input/deepfake-and-real-images')


Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [ ]:
print("Class names and their labels:", train.class_names)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

model = Sequential([
    Input((256, 256, 3)),
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1),
    RandomContrast(0.1),

    Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    GlobalAveragePooling2D(),

    Dense(512, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])


In [11]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)              │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip_1 (RandomFlip)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_1 (RandomRotation)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_1 (RandomZoom)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation_1                 │ (None, 256, 256, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast_1 (RandomContrast)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 256, 256, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 128, 128, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_18 (LeakyReLU)           │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_19 (LeakyReLU)           │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 10,551,265 (40.25 MB)

 Trainable params: 10,543,073 (40.22 MB)

 Non-trainable params: 8,192 (32.00 KB)

In [12]:
from tensorflow.keras.optimizers import Adam

custom_adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint(filepath='best_model.keras',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

earlystop=EarlyStopping(monitor='val_accuracy',patience=5,verbose=1,restore_best_weights=True)

lr_scheduler=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


model.fit(train,epochs=20,validation_data=val,callbacks=[checkpoint, earlystop, lr_scheduler])


Epoch 1/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.7306 - loss: 0.6615
Epoch 1: val_accuracy improved from -inf to 0.82188, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 415s 181ms/step - accuracy: 0.7306 - loss: 0.6614 - val_accuracy: 0.8219 - val_loss: 0.5350 - learning_rate: 0.0010
Epoch 2/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.9237 - loss: 0.2651
Epoch 2: val_accuracy improved from 0.82188 to 0.88021, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 396s 181ms/step - accuracy: 0.9237 - loss: 0.2651 - val_accuracy: 0.8802 - val_loss: 0.3719 - learning_rate: 0.0010
Epoch 3/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.9398 - loss: 0.2168
Epoch 3: val_accuracy did not improve from 0.88021
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 395s 181ms/step - accuracy: 0.9398 - loss: 0.2168 - val_accuracy: 0.8396 - val_loss: 0.4283 - learning_rate: 0.0010
Epoch 4/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accur

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

y_true = []
y_pred = []

for images, labels in test:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype("int32").flatten())  

y_true = np.array(y_true)
y_pred = np.array(y_pred)

test_loss, test_accuracy = model.evaluate(test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
2/2 ━━━━━━━━

In [18]:
!zip -r folder.zip /kaggle/working/best_model.keras

updating: kaggle/working/best_model.keras (deflated 8%)


In [19]:
from IPython.display import FileLink

FileLink(r'folder.zip')

/kaggle/working/folder.zip